In [2]:
import requests
import pandas as pd
import time

In [3]:
key = "RGAPI-c6392c67-5223-4bd1-a76a-0fa0d8c30a7c"
summonerName = "Bytaizer"
nb_games = 10

## Récup du PUUID

In [4]:
recup_puuid = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}?api_key={key}"
reponse_puuid = requests.get(recup_puuid)
res_puuid=reponse_puuid.json()
puuid = res_puuid["puuid"]

## A partir du PUUID, on récup jusqu'à 100 games

In [112]:
recup_games = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={nb_games}&api_key={key}"
reponse_games = requests.get(recup_games)
res_games=reponse_games.json()

## Création d'un Dataframe

### Fonction pour faciliter l'accession aux infos de ['info'][['participants']

In [9]:
def donnees_part(data,key):
    """ la fonction permet de récupérer des valeurs du dictionnaire ayant comme clé ['info']['participant'] par equipe
    data est le dictionnaire des participants"""
    L1,L2=[],[] # création de listes pours les données des team 1 et 2 (pour une des games)
    for participant in data:
        if len(L1) <5:# une fois la première équipe remplie, on remplie la deuxième
            L1.append(participant[key])
        else :
            L2.append(participant[key])
    return(L1,L2)
    

In [10]:
def recup_donnees_team(keys,data,dict_final):
    """permet de récup les données par team (de ['info']['participants']) et de les implémenter dans un dictionnaire"""
    dict=data['info']['participants']
    for key in keys :
        x=donnees_part(dict,key)
        dict_final[key+"1"].append(x[0])
        dict_final[key+"2"].append(x[1])
    return(dict_final)

In [94]:
gameDuration,gameMode,win = [],[],[]
keys = ['championName','championId','champExperience','champLevel'] #clés des informations qu'il faut trier par équipe
dict_final = {keys[i]+ str(a): [] for i in range(len(keys)) for a in (1, 2)} #création du dictionnaire vide pour les infos par team : [champExperience1 : [], champExperience2 : [],

for games in res_games : #on parcours l'ensemble des parties de la liste
    api_url =f"https://europe.api.riotgames.com/lol/match/v5/matches/{games}?api_key={key}"
    response = requests.get(api_url)
    data = response.json()
    
    dict_final=recup_donnees_team(keys,data,dict_final)

    gameDuration.append(data['info']['gameDuration'])
    gameMode.append(data['info']['gameMode'])
    win.append(1 if data['info']['participants'][0]['win']==True else 2) #permet de savoir quelle équipe a gagné (on regarde pour le premier joueur)
    
dict2 = {'GameId': res_games, 'GameDuration' : gameDuration, 'gameMode': gameMode,'win':win}
dict2.update(dict_final) #on concatène ensuite les dictionnaires
data_info = pd.DataFrame(dict2)

In [92]:
#print(data_info)

In [95]:
data_info.to_csv("infos_games.csv ")

## Pagination et problèmes de quotas

In [5]:
def bcp_demandes(nb_games):
    if nb_games >100 :
        recup_games = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={100}&api_key={key}"
        nb_games = nb_games - 100
        reponse_games = requests.get(recup_games)
        res_games=reponse_games.json()
    if response.status_code == 429:
        print("trop de demandes")
        time.sleep(int(reponse_games.headers["Retry-After"]))
        print(reponse_games.headers["Retry-After"])
    print(res_games)



### Si on demande plus de 100 games :

In [6]:
nb_games = 300

In [7]:
res_games = []
while nb_games >100:
    recup_games = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={100}&api_key={key}"
    reponse_games = requests.get(recup_games)
    res_games = res_games + reponse_games.json()
    nb_games-=100
    
recup_games = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={nb_games}&api_key={key}"
reponse_games = requests.get(recup_games)
res_games = res_games + reponse_games.json()
#print(res_games)

In [11]:
gameDuration,gameMode,win = [],[],[]
keys = ['championName','championId','champExperience','champLevel'] #clés des informations qu'il faut trier par équipe
dict_final = {keys[i]+ str(a): [] for i in range(len(keys)) for a in (1, 2)} #création du dictionnaire vide pour les infos par team : [champExperience1 : [], champExperience2 : [],

i=0

for games in res_games : #on parcours l'ensemble des parties de la liste
    api_url =f"https://europe.api.riotgames.com/lol/match/v5/matches/{games}?api_key={key}"
    response = requests.get(api_url)
    
    if response.status_code == 429:
        print("trop de demandes")
        time.sleep(int(response.headers["Retry-After"])+2) # +2 car sinon le time.sleep semblait être un tout petit peu trop court
        response = requests.get(api_url)
    
    data = response.json()
    
    dict_final=recup_donnees_team(keys,data,dict_final)

    gameDuration.append(data['info']['gameDuration'])
    gameMode.append(data['info']['gameMode'])
    win.append(1 if data['info']['participants'][0]['win']==True else 2) #permet de savoir quelle équipe a gagné (on regarde pour le premier joueur)
    
dict2 = {'GameId': res_games, 'GameDuration' : gameDuration, 'gameMode': gameMode,'win':win}
dict2.update(dict_final) #on concatène ensuite les dictionnaires
data_info = pd.DataFrame(dict2)

trop de demandes
trop de demandes


In [12]:
data_info.to_csv("infos_games.csv ")

## Premiers essais Machine Learning (j'ai un problème de type si je récupère la même chose à partir du csv dans un autre notebook)

#### On split les colonnes pour obtenir quelque chose de traitable

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error # pour importer la fonction de la MAE
from sklearn.model_selection import train_test_split

In [12]:
data2 = data_info[["championId1","championId2","win"]] 

In [13]:
pd.options.mode.chained_assignment = None  # default='warn' on enlève le warning
data2[['1', '2','3','4','5']] = pd.DataFrame(data2.championId1.tolist(), index= data2.index)
data2[['6','7','8','9','10']] = pd.DataFrame(data2.championId2.tolist(), index= data2.index)
data2.drop(columns=["championId1","championId2"])

,win,1,2,3,4,5,6,7,8,9,10
0,2,134,96,517,555,11,18,98,166,517,45
1,2,86,20,38,99,84,22,202,126,115,79
2,1,161,120,48,91,238,45,143,22,119,53
3,2,81,20,25,45,145,72,12,360,45,497
4,1,777,22,115,222,74,106,240,238,887,64
...,...,...,...,...,...,...,...,...,...,...,...
295,2,887,11,777,895,555,122,245,4,15,25
296,2,58,77,112,81,99,122,141,163,29,25
297,2,122,77,61,51,101,19,56,31,21,143
298,1,122,11,517,51,201,62,113,134,119,53


In [63]:
modele = RandomForestRegressor(n_estimators=200)

In [18]:
X = data2[['1', '2','3','4','5','6','7','8','9','10']]
y = data2["win"]

In [19]:
train_X, val_X, train_y, val_y = train_test_split(X, y,train_size=0.8, test_size=0.2, random_state=1)

In [81]:
modele.fit(train_X,train_y) # pour entrainer le modèle
donnees_predites = modele.predict(val_X)

In [ ]:
mean_absolute_error(val_y, donnees_predites)

0.10616666666666666

In [100]:
predictions = pd.DataFrame({"prédictions" : donnees_predites}) # on compare en regardant les valeurs pour que ce soit plus sympa que la MAE
val_y.reset_index(drop=True, inplace=True)
predictions.reset_index(drop=True, inplace=True)
comparaison = pd.concat( [predictions, val_y], axis=1)
print(comparaison.head())

   prédictions  win
0        1.940    2
1        1.120    1
2        1.160    1
3        1.095    1
4        1.945    2


#### On essaye d'être un peu plus précis avec de la cross-validation

In [102]:
## cross validation

scores = -1 * cross_val_score(RandomForestRegressor(n_estimators=200), X, y,cv=5, scoring='neg_mean_absolute_error')
print(scores)

[0.06308333 0.06158333 0.06383333 0.06783333 0.06233333]


#### A la recherche du meilleur nombre d'arbre (résultats pas hyper concluants)

In [20]:
from sklearn.model_selection import cross_val_score
import statistics

def recherche_RFR(max,pas,min=10):
    min_mean = 10 #recherche du min des cross évaluations, 10 au début car les moyennnes seront entre 0 et 2
    modele=0 #initialisation du meilleur modele
    for i in range(min,max,pas):
        scores = -1 * cross_val_score(RandomForestRegressor(n_estimators=i), X, y,cv=5, scoring='neg_mean_absolute_error')
        if statistics.mean(scores)< min_mean :
            modele = RandomForestRegressor(n_estimators=i)
            min_mean = statistics.mean(scores)
    return (modele)

best_modele = recherche_RFR(1000,100)
print(best_modele)


KeyboardInterrupt



### test rapide avec XGBoost

In [27]:
X = data2[['1', '2','3','4','5','6','7','8','9','10']]
y = data2["win"]

In [28]:
train_X, val_X, train_y, val_y = train_test_split(X, y,train_size=0.8, test_size=0.2, random_state=1)

In [29]:
from xgboost import XGBRegressor

modele = XGBRegressor(n_estimators = 200, learning_rate=0.05, n_jobs=4) #bcp de paramètres dont n_estimators qui donne le fois que le cycle est fait = nombre de modèles dans l'ensemble
                                                                # voir utilisation précise learning_rate : multiplication des res de chaque modèle de l'ensemble par un petit nombre
                                                                #n_jobs : = au nb de coeur du PC, pour // les calculs (utile seulement si grd Dataset)
modele.fit(train_X, train_y, 
             early_stopping_rounds=5, #si l'erreur se détériore sur 5 cycles on arrête le programme même si < n_estimators
             eval_set=[(val_X, val_y)], #obligatoire quand early_stopping_rounds utilisé
             verbose=False)

donnees_predites = modele.predict(val_X)

mean_absolute_error(val_y, donnees_predites)


C:\Users\maila\anaconda3\envs\Projet\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.004452734192212423

In [30]:
scores = -1 * cross_val_score(XGBRegressor(n_estimators = 200, learning_rate=0.05, n_jobs=4), X, y,cv=5, scoring='neg_mean_absolute_error')
print(scores)

[0.00108672 0.00145781 0.00132565 0.00221002 0.00125324]


In [25]:
predictions = pd.DataFrame({"prédictions" : donnees_predites}) # on compare en regardant les valeurs pour que ce soit plus sympa que la MAE
val_y.reset_index(drop=True, inplace=True)
predictions.reset_index(drop=True, inplace=True)
comparaison = pd.concat( [predictions, val_y], axis=1)
print(comparaison.head())

   prédictions  win
0     1.997136    2
1     1.006137    1
2     1.002926    1
3     1.001536    1
4     1.998437    2
